# Preprocessing Data

In [ ]:
# mount google drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# import dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

In [ ]:
# read in the TB_Data_2000_2022_total_count.csv
df = pd.read_csv("/content/drive/My Drive/project4/resources/TB_Data_2000_2022_total_count.csv")
df

,Indicator,Year,Geography,FIPS,Cases,Rate per 100000
0,Tuberculosis,2000,Alabama,1,310,7.0
1,Tuberculosis,2001,Alabama,1,264,5.9
2,Tuberculosis,2002,Alabama,1,233,5.2
3,Tuberculosis,2003,Alabama,1,258,5.7
4,Tuberculosis,2004,Alabama,1,211,4.7
...,...,...,...,...,...,...
1168,Tuberculosis,2018,Wyoming,56,1,0.2
1169,Tuberculosis,2019,Wyoming,56,1,0.2
1170,Tuberculosis,2020 (COVID-19 Pandemic),Wyoming,56,0,0.0
1171,Tuberculosis,2021,Wyoming,56,3,0.5


In [ ]:
# check the data types in the aggregate case count dataset
df.dtypes

Indicator           object
Year                object
Geography           object
FIPS                 int64
Cases               object
Rate per 100000    float64
dtype: object

In [ ]:
# update "2020 (COVID-19 Pandemic)" under the Year column to just the year
df.loc[df['Year'] == '2020 (COVID-19 Pandemic)', 'Year'] = '2020'

In [ ]:
# convert the Year and Cases columns' data type to integer
df[['Year','Cases']] = df.replace(',', '', regex=True)[['Year','Cases']].astype(int)

In [ ]:
# divide the years into pre- and post-covid

# first create a function to categorize pre-/post-covid
def covid(year):
    if year < 2020:
        return 'Pre'
    else:
        return 'Post'

# apply the function to the Year column using the apply() function
df['COVID'] = df['Year'].apply(covid)
df

,Indicator,Year,Geography,FIPS,Cases,Rate per 100000,COVID
0,Tuberculosis,2000,Alabama,1,310,7.0,Pre
1,Tuberculosis,2001,Alabama,1,264,5.9,Pre
2,Tuberculosis,2002,Alabama,1,233,5.2,Pre
3,Tuberculosis,2003,Alabama,1,258,5.7,Pre
4,Tuberculosis,2004,Alabama,1,211,4.7,Pre
...,...,...,...,...,...,...,...
1168,Tuberculosis,2018,Wyoming,56,1,0.2,Pre
1169,Tuberculosis,2019,Wyoming,56,1,0.2,Pre
1170,Tuberculosis,2020,Wyoming,56,0,0.0,Post
1171,Tuberculosis,2021,Wyoming,56,3,0.5,Post


In [ ]:
# drop the Indicator, FIPS and Rate columns
df = df.drop(columns=['Indicator', 'FIPS', 'Rate per 100000'])

In [ ]:
# convert the categorical data (Year and State) to numeric with 'pd.get_dummies'
numeric = pd.get_dummies(df, dtype=int)
numeric

,Year,Cases,Geography_Alabama,Geography_Alaska,Geography_Arizona,Geography_Arkansas,Geography_California,Geography_Colorado,Geography_Connecticut,Geography_Delaware,...,Geography_Texas,Geography_Utah,Geography_Vermont,Geography_Virginia,Geography_Washington,Geography_West Virginia,Geography_Wisconsin,Geography_Wyoming,COVID_Post,COVID_Pre
0,2000,310,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1,2001,264,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
2,2002,233,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
3,2003,258,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
4,2004,211,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1168,2018,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,1
1169,2019,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,1
1170,2020,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,1,0
1171,2021,3,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,1,0


In [ ]:
# filter pre- and post-COVID data
pre_covid_data = numeric[numeric['COVID_Pre'] == 1]
post_covid_data = numeric[numeric['COVID_Post'] == 1]

# pre-COVID data
X_pre_covid = pre_covid_data.drop(['Cases'], axis=1)
y_pre_covid = pre_covid_data['Cases']

# post-COVID data
X_post_covid = post_covid_data.drop(['Cases'], axis=1)
y_post_covid = post_covid_data['Cases']

In [ ]:
# split pre-COVID data into training and testing datasets
X_train_pre_covid, X_test_pre_covid, y_train_pre_covid, y_test_pre_covid = train_test_split(X_pre_covid, y_pre_covid, random_state=1)

# split post-COVID data into training and testing datasets
X_train_post_covid, X_test_post_covid, y_train_post_covid, y_test_post_covid = train_test_split(X_post_covid, y_post_covid, random_state=1)

In [ ]:
# create a StandardScaler instances
scaler = StandardScaler()

# fit the StandardScaler on pre-COVID (training) data
X_scaler = scaler.fit(X_train_pre_covid)

# transform the training and testing data using the fitted scaler
X_train_pre_covid_scaled = scaler.transform(X_train_pre_covid)
X_test_pre_covid_scaled = scaler.transform(X_test_pre_covid)
X_train_post_covid_scaled = scaler.transform(X_train_post_covid)
X_test_post_covid_scaled = scaler.transform(X_test_post_covid)

# Compile, Train and Evaluate the Model

In [ ]:
# define the model
input_features = len(X_train_pre_covid_scaled[0])
nodes1 = 100
nodes2 = 100

nn_model = tf.keras.models.Sequential()

# first hidden layer
nn_model.add(tf.keras.layers.Dense(units=nodes1, input_dim=input_features, activation='tanh'))

# second hidden layer
nn_model.add(tf.keras.layers.Dense(units=nodes2, activation='relu'))

# output layer
nn_model.add(tf.keras.layers.Dense(units=1))

# check the structure of the model
nn_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 100)               5500      
                                                                 
 dense_1 (Dense)             (None, 100)               10100     
                                                                 
 dense_2 (Dense)             (None, 1)                 101       
                                                                 
Total params: 15701 (61.33 KB)
Trainable params: 15701 (61.33 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [ ]:
# compile the model
nn_model.compile(optimizer='adam', loss='mean_squared_error', metrics=['accuracy'])

In [ ]:
# train the model
fit_model = nn_model.fit(X_train_pre_covid_scaled, y_train_pre_covid, epochs=50)

Epoch 1/50
24/24 [==============================] - 1s 2ms/step - loss: 226817.4219 - accuracy: 0.0026 
Epoch 2/50
24/24 [==============================] - 0s 3ms/step - loss: 225429.7969 - accuracy: 0.0026
Epoch 3/50
24/24 [==============================] - 0s 2ms/step - loss: 222542.9062 - accuracy: 0.0026
Epoch 4/50
24/24 [==============================] - 0s 2ms/step - loss: 216062.8281 - accuracy: 0.0026
Epoch 5/50
24/24 [==============================] - 0s 2ms/step - loss: 201401.1406 - accuracy: 0.0026
Epoch 6/50
24/24 [==============================] - 0s 2ms/step - loss: 178159.4375 - accuracy: 0.0026
Epoch 7/50
24/24 [==============================] - 0s 2ms/step - loss: 151227.4531 - accuracy: 0.0026
Epoch 8/50
24/24 [==============================] - 0s 2ms/step - loss: 127353.6406 - accuracy: 0.0013
Epoch 9/50
24/24 [==============================] - 0s 2ms/step - loss: 110382.0000 - accuracy: 0.0026
Epoch 10/50
24/24 [==============================] - 0s 2ms/step - loss:

In [ ]:
# evaluate the model using the test data
model_loss, model_accuracy = nn_model.evaluate(X_test_post_covid_scaled,y_test_post_covid,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

2/2 - 0s - loss: 9227.4463 - accuracy: 0.0000e+00 - 237ms/epoch - 119ms/step
Loss: 9227.4462890625, Accuracy: 0.0
